In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, random_split, Subset, Dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np
import copy
from tqdm import tqdm
from scipy.stats import mode
import torch
import gc
import pandas as pd
from sklearn.model_selection import cross_val_score
import torch.nn.functional as F
import random





In [3]:
def save_model(model,model_name):

    torch.save(model.state_dict(), f'/scratch/{model_name}.pth')


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def clear_CUDA():

    # Clear CUDA cache
    torch.cuda.empty_cache()

    # Run garbage collector
    gc.collect()

clear_CUDA()

In [5]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [6]:
def create_shards(dataset, num_shards):
    shard_size = len(dataset) // num_shards
    shards = [data.Subset(dataset, range(i * shard_size, (i + 1) * shard_size)) for i in range(num_shards)]
    return shards

def create_slices(shard, num_slices):
    slice_size = len(shard) // num_slices
    slices = [data.Subset(shard, range(i * slice_size, (i + 1) * slice_size)) for i in range(num_slices)]
    return slices


In [7]:

class ModifiedResNet18(nn.Module):
    shared_resnet = models.resnet18(pretrained=True)
    shared_resnet.fc = nn.Identity()
    for param in shared_resnet.parameters():
        param.requires_grad = False

    def __init__(self, num_classes=10, shared=True):
        super(ModifiedResNet18, self).__init__()

        if shared:
            self.resnet = ModifiedResNet18.shared_resnet
        else:
            self.resnet = models.resnet18(pretrained=True)
            self.resnet.fc = nn.Identity()

        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc(x)
        return x

# model1 = ModifiedResNet18(num_classes=10)
# model2 = ModifiedResNet18(num_classes=10)

def train_model(model, dataloaders, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0
        total = 0
        for inputs, labels in tqdm(dataloaders, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total
        # print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
    return model


/projects/envs/conda/nmoradi/envs/temp_env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/projects/envs/conda/nmoradi/envs/temp_env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
def sisa_train(shards, num_slices, num_epochs, initial_model, criterion, optimizer_fn):
    num_shards = len(shards)
    slice_models = {shard: [] for shard in range(len(shards))}
    # slice_models = []
    
    for shard_idx, shard in enumerate(shards):
        slices = create_slices(shard, num_slices)
        # model = copy.deepcopy(initial_model)
        for slice_idx, slice_ in enumerate(slices):
            model = copy.deepcopy(initial_model)
            dataloader = data.DataLoader(slice_, batch_size=64, shuffle=True)
            optimizer = optimizer_fn(model.parameters())
            print(f"Training shard {shard_idx+1}/{len(shards)}, slice {slice_idx+1}/{num_slices}")
            model = train_model(model, dataloader, criterion, optimizer, num_epochs)
            # slice_models.append(model)
            # model_name = f'{num_shards}{num_slices}_{shard_idx}{slice_idx}'
            # save_model(model,model_name)
            slice_models[shard_idx].append(model)
        


    return slice_models #, shard_preds_MV, shard_preds_WA, final_preds_MV, final_preds_WA

In [9]:
def sisa_unlearn(shards, num_slices, shard_models, initial_model, criterion, optimizer_fn, data_to_forget):
    num_shards = len(shards)
    
    shard_models_updated = {shard: [] for shard in range(len(shards))}
    # shard_models_updated = []
    for shard_idx, shard in enumerate(shards):
        slices = create_slices(shard, num_slices)
        local_indices_to_forget = [shard.indices.index(idx) for idx in data_to_forget if idx in shard.indices]
        
        slices_to_update = [i for i, slice_ in enumerate(slices) if any(idx in slice_.indices for idx in local_indices_to_forget)]
        
        if slices_to_update:
            models_shard = []
            print(f"Retraining shard {shard_idx+1}/{len(shards)} containing data to forget")

            for slice_idx, slice_ in enumerate(slices):
                if slice_idx in slices_to_update:
                    model = copy.deepcopy(initial_model)
                    updated_indices = [idx for idx in slice_.indices if shard.indices[idx] not in local_indices_to_forget]
                    data_refined = data.Subset(shard, updated_indices)
                    dataloader = data.DataLoader(data_refined, batch_size=64, shuffle=True)
                    optimizer = optimizer_fn(model.parameters())
                    model = train_model(model, dataloader, criterion, optimizer, num_epochs=5)
                    # model_name = f'{num_shards}{num_slices}_{shard_idx}{slice_idx}_unlearned'
                    # save_model(model,model_name)
                    print(f"Slice {slice_idx+1}/{len(slices)} retrained for shard {shard_idx+1}/{len(shards)}")
                else:
                    model = shard_models[shard_idx][slice_idx]
                models_shard.append(model)
        else:
            models_shard = copy.deepcopy(shard_models[shard_idx])
        shard_models_updated[shard_idx] = models_shard
    
    return shard_models_updated

In [10]:
def majority_voting_slices(models, dataloader):
    all_preds = []
    for shard in models.keys():
        models_shard = models[shard]
        for i, model in enumerate(models_shard):
            model.eval()
            preds = []
            with torch.no_grad():
                for inputs, _ in tqdm(dataloader, desc=f"Evaluating model: Shard {shard + 1}/{len(models.keys())}, slice {i + 1}/{len(models_shard)}"):
                    inputs = inputs.cuda()
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs, 1)
                    preds.extend(predicted.cpu().numpy())
            all_preds.append(preds)
    all_preds = np.array(all_preds)
    majority_preds = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=all_preds)
    
    return majority_preds

def aggregate_average(models, test_loader):
    all_predictions = []
    for shard in models.keys():
        models_shard = models[shard]
        for i, model in enumerate(models_shard):
            model.eval()  # Set model to evaluation mode
            model_predictions = []
            for inputs, _ in tqdm(test_loader, desc=f"Evaluating model: Shard {shard + 1}/{len(models.keys())}, slice {i + 1}/{len(models_shard)}"):
                inputs = inputs.to(device)
                with torch.no_grad():
                    pred = model(inputs)
                model_predictions.append(pred.cpu())
            all_predictions.append(torch.cat(model_predictions))
        
    averaged_probabilities = torch.mean(torch.stack(all_predictions), dim=0)
    aggregated_predictions = torch.argmax(averaged_probabilities, dim=1)
    averaged_probabilities = F.softmax(averaged_probabilities, dim=1)
    return aggregated_predictions, averaged_probabilities

def weighted_averaging(models, dataloader):
    weights = [1 / len(models)] * len(models)
    all_outputs = []
    for model in models:
        model.eval()
        outputs = []
        with torch.no_grad():
            for inputs, _ in tqdm(dataloader, desc="Weighted Averaging"):
                inputs = inputs.cuda()
                output = model(inputs)
                outputs.extend(output.cpu().numpy())
        all_outputs.append(outputs)
    all_outputs = np.array(all_outputs)
    weighted_outputs = np.average(all_outputs, axis=0, weights=weights)
    weighted_outputs = np.exp(weighted_outputs) / np.sum(np.exp(weighted_outputs), axis=1, keepdims=True)  # Softmax normalization
    return weighted_outputs

In [11]:
def evaluate_models(models, dataloader):
    targets = []
    for _, labels in dataloader:
        targets.extend(labels.numpy())

    # final_preds = majority_voting_slices(models, dataloader)
    final_preds, _ = aggregate_average(models, dataloader)

    # print(final_preds)
    # print(targets)
    
    accuracy = accuracy_score(targets, final_preds)
    precision = precision_score(targets, final_preds, average='macro')
    recall = recall_score(targets, final_preds, average='macro')
    f1 = f1_score(targets, final_preds, average='macro')
    auroc = roc_auc_score(targets, np.eye(len(np.unique(targets)))[final_preds], multi_class='ovr')
    return accuracy, precision, recall, f1, auroc

In [12]:
shard_counts = [5, 10, 20]
slice_counts = [5, 10, 20]
num_epochs = 5

best_accuracy = 0       
best_model = None        
best_result = {}      

best_accuracy_unlearned = 0      
best_model_unlearned = None        
best_result_unlearned = {}         

initial_model = ModifiedResNet18(shared=False).cuda()
criterion = nn.CrossEntropyLoss()
optimizer_fn = lambda params: optim.Adam(params, lr=0.001)

results = []
results_unlearning = []
train_data, val_data = train_test_split(train_dataset, test_size=0.2, random_state=42)
val_loader = data.DataLoader(val_data, batch_size=64, shuffle=False)


In [13]:
S = 5
R = 5
shards = create_shards(train_dataset, S)
shard_models = sisa_train(shards, R, num_epochs, initial_model, criterion, optimizer_fn)


test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

accuracy, precision, recall, f1, auroc = evaluate_models(shard_models, test_loader)
results.append((S, R, "Majority Voting", accuracy, precision, recall, f1, auroc))

if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_model = shard_models
    best_result = (S, R, accuracy, precision, recall, f1, auroc)


Training shard 1/5, slice 1/5


Epoch 1/5:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch 5/5: 100%|██████████| 32/32 [00:00<00:00, 35.34it/s]


Training shard 1/5, slice 2/5


Epoch 5/5: 100%|██████████| 32/32 [00:00<00:00, 35.35it/s]


...

Epoch 5/5: 100%|██████████| 32/32 [00:00<00:00, 35.24it/s]


Training shard 5/5, slice 4/5


Epoch 5/5: 100%|██████████| 32/32 [00:00<00:00, 35.25it/s]


Training shard 5/5, slice 5/5


Evaluating model: Shard 5/5, slice 5/5: 100%|██████████| 157/157 [00:02<00:00, 53.96it/s]


In [14]:

# Perform unlearning
data_to_forget_indices = np.random.choice(len(train_data), 500, replace=False)
shard_models_unlearned = sisa_unlearn(shards, R, shard_models, initial_model, criterion, optimizer_fn, data_to_forget_indices)

accuracy, precision, recall, f1, auroc = evaluate_models(shard_models_unlearned, test_loader)
results_unlearning.append((S, R, "Majority Voting" + " (Unlearned)", accuracy, precision, recall, f1, auroc))

        
if accuracy > best_accuracy_unlearned:
    best_accuracy_unlearned = accuracy
    best_model_unlearned = shard_models_unlearned
    best_result_unlearned = (S, R, accuracy, precision, recall, f1, auroc)

clear_CUDA()

Retraining shard 1/5 containing data to forget


Epoch 5/5: 100%|██████████| 31/31 [00:00<00:00, 35.16it/s]


Slice 1/5 retrained for shard 1/5


Epoch 5/5: 100%|██████████| 31/31 [00:00<00:00, 35.26it/s]


Slice 2/5 retrained for shard 1/5


...

Epoch 5/5: 100%|██████████| 32/32 [00:00<00:00, 35.52it/s]


Slice 4/5 retrained for shard 4/5


Epoch 5/5: 100%|██████████| 32/32 [00:00<00:00, 35.47it/s]


Slice 5/5 retrained for shard 4/5


Evaluating model: Shard 5/5, slice 5/5: 100%|██████████| 157/157 [00:02<00:00, 54.04it/s]


In [15]:
S = 5
R = 10
shards = create_shards(train_dataset, S)
shard_models = sisa_train(shards, R, num_epochs, initial_model, criterion, optimizer_fn)

test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

accuracy, precision, recall, f1, auroc = evaluate_models(shard_models, test_loader)
results.append((S, R, "Majority Voting", accuracy, precision, recall, f1, auroc))

if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_model = shard_models
    best_result = (S, R, accuracy, precision, recall, f1, auroc)

# Perform unlearning
data_to_forget_indices = np.random.choice(len(train_data), 500, replace=False)
shard_models_unlearned = sisa_unlearn(shards, R, shard_models, initial_model, criterion, optimizer_fn, data_to_forget_indices)

accuracy, precision, recall, f1, auroc = evaluate_models(shard_models_unlearned, test_loader)
results_unlearning.append((S, R, "Majority Voting" + " (Unlearned)", accuracy, precision, recall, f1, auroc))


if accuracy > best_accuracy_unlearned:
    best_accuracy_unlearned = accuracy
    best_model_unlearned = shard_models_unlearned
    best_result_unlearned = (S, R, accuracy, precision, recall, f1, auroc)

clear_CUDA()

Training shard 1/5, slice 1/10


Epoch 1/5:  25%|██▌       | 4/16 [00:00<00:00, 32.83it/s]

Epoch 5/5: 100%|██████████| 16/16 [00:00<00:00, 35.48it/s]


Training shard 1/5, slice 2/10


Epoch 5/5: 100%|██████████| 16/16 [00:00<00:00, 35.41it/s]


...

Epoch 5/5: 100%|██████████| 16/16 [00:00<00:00, 35.70it/s]


Slice 9/10 retrained for shard 4/5


Epoch 5/5: 100%|██████████| 16/16 [00:00<00:00, 35.83it/s]


Slice 10/10 retrained for shard 4/5


Evaluating model: Shard 5/5, slice 10/10: 100%|██████████| 157/157 [00:02<00:00, 53.83it/s]


In [16]:
S = 5
R = 20
shards = create_shards(train_dataset, S)
shard_models = sisa_train(shards, R, num_epochs, initial_model, criterion, optimizer_fn)

test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

accuracy, precision, recall, f1, auroc = evaluate_models(shard_models, test_loader)
results.append((S, R, "Majority Voting", accuracy, precision, recall, f1, auroc))

if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_model = shard_models
    best_result = (S, R, accuracy, precision, recall, f1, auroc)


# Perform unlearning
data_to_forget_indices = np.random.choice(len(train_data), 500, replace=False)
shard_models_unlearned = sisa_unlearn(shards, R, shard_models, initial_model, criterion, optimizer_fn, data_to_forget_indices)

accuracy, precision, recall, f1, auroc = evaluate_models(shard_models_unlearned, test_loader)
results_unlearning.append((S, R, "Majority Voting" + " (Unlearned)", accuracy, precision, recall, f1, auroc))

if accuracy > best_accuracy_unlearned:
    best_accuracy_unlearned = accuracy
    best_model_unlearned = shard_models_unlearned
    best_result_unlearned = (S, R, accuracy, precision, recall, f1, auroc)
    
clear_CUDA()

Training shard 1/5, slice 1/20


Epoch 5/5: 100%|██████████| 8/8 [00:00<00:00, 35.63it/s]


Training shard 1/5, slice 2/20


Epoch 5/5: 100%|██████████| 8/8 [00:00<00:00, 35.56it/s]


Training shard 1/5, slice 3/20


...

Epoch 5/5: 100%|██████████| 8/8 [00:00<00:00, 36.33it/s]


Slice 19/20 retrained for shard 4/5


Epoch 5/5: 100%|██████████| 8/8 [00:00<00:00, 38.66it/s]


Slice 20/20 retrained for shard 4/5


Evaluating model: Shard 5/5, slice 20/20: 100%|██████████| 157/157 [00:02<00:00, 53.71it/s]


In [17]:
S = 10
R = 5
shards = create_shards(train_dataset, S)
shard_models = sisa_train(shards, R, num_epochs, initial_model, criterion, optimizer_fn)

test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

accuracy, precision, recall, f1, auroc = evaluate_models(shard_models, test_loader)
results.append((S, R, "Majority Voting", accuracy, precision, recall, f1, auroc))

if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_model = shard_models
    best_result = (S, R, accuracy, precision, recall, f1, auroc)
    
# Perform unlearning
data_to_forget_indices = np.random.choice(len(train_data), 500, replace=False)
shard_models_unlearned = sisa_unlearn(shards, R, shard_models, initial_model, criterion, optimizer_fn, data_to_forget_indices)

accuracy, precision, recall, f1, auroc = evaluate_models(shard_models_unlearned, test_loader)
results_unlearning.append((S, R, "Majority Voting" + " (Unlearned)", accuracy, precision, recall, f1, auroc))


if accuracy > best_accuracy_unlearned:
    best_accuracy_unlearned = accuracy
    best_model_unlearned = shard_models_unlearned
    best_result_unlearned = (S, R, accuracy, precision, recall, f1, auroc)

clear_CUDA()

Training shard 1/10, slice 1/5


Epoch 3/5:   0%|          | 0/16 [00:00<?, ?it/s]

Epoch 5/5: 100%|██████████| 16/16 [00:00<00:00, 35.96it/s]


Training shard 1/10, slice 2/5


Epoch 5/5: 100%|██████████| 16/16 [00:00<00:00, 35.60it/s]


...

Epoch 5/5: 100%|██████████| 16/16 [00:00<00:00, 35.17it/s]


Slice 4/5 retrained for shard 8/10


Epoch 5/5: 100%|██████████| 16/16 [00:00<00:00, 35.04it/s]


Slice 5/5 retrained for shard 8/10


Evaluating model: Shard 10/10, slice 5/5: 100%|██████████| 157/157 [00:02<00:00, 53.26it/s]


In [18]:
S = 10
R = 10
shards = create_shards(train_dataset, S)
shard_models = sisa_train(shards, R, num_epochs, initial_model, criterion, optimizer_fn)

test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

accuracy, precision, recall, f1, auroc = evaluate_models(shard_models, test_loader)
results.append((S, R, "Majority Voting", accuracy, precision, recall, f1, auroc))

if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_model = shard_models
    best_result = (S, R, accuracy, precision, recall, f1, auroc)

# Perform unlearning
data_to_forget_indices = np.random.choice(len(train_data), 500, replace=False)
shard_models_unlearned = sisa_unlearn(shards, R, shard_models, initial_model, criterion, optimizer_fn, data_to_forget_indices)

accuracy, precision, recall, f1, auroc = evaluate_models(shard_models_unlearned, test_loader)
results_unlearning.append((S, R, "Majority Voting" + " (Unlearned)", accuracy, precision, recall, f1, auroc))

if accuracy > best_accuracy_unlearned:
    best_accuracy_unlearned = accuracy
    best_model_unlearned = shard_models_unlearned
    best_result_unlearned = (S, R, accuracy, precision, recall, f1, auroc)

clear_CUDA()

Training shard 1/10, slice 1/10


Epoch 5/5: 100%|██████████| 8/8 [00:00<00:00, 34.64it/s]


Training shard 1/10, slice 2/10


Epoch 3/5:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 5/5: 100%|██████████| 8/8 [00:00<00:00, 34.64it/s]


...

Epoch 5/5: 100%|██████████| 8/8 [00:00<00:00, 35.44it/s]


Slice 9/10 retrained for shard 8/10


Epoch 5/5: 100%|██████████| 8/8 [00:00<00:00, 35.42it/s]


Slice 10/10 retrained for shard 8/10


Evaluating model: Shard 10/10, slice 10/10: 100%|██████████| 157/157 [00:02<00:00, 53.77it/s]


In [19]:
clear_CUDA()

In [20]:
# S = 10
# R = 20
# shards = create_shards(train_dataset, S)
# shard_models = sisa_train(shards, R, num_epochs, initial_model, criterion, optimizer_fn)

# test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# accuracy, precision, recall, f1, auroc = evaluate_models(shard_models, test_loader)
# results.append((S, R, "Majority Voting", accuracy, precision, recall, f1, auroc))

# if accuracy > best_accuracy:
#     best_accuracy = accuracy
#     best_model = shard_models
#     best_result = (S, R, accuracy, precision, recall, f1, auroc)

# # Perform unlearning
# data_to_forget_indices = np.random.choice(len(train_data), 500, replace=False)
# shard_models_unlearned = sisa_unlearn(shards, R, shard_models, initial_model, criterion, optimizer_fn, data_to_forget_indices)

# accuracy, precision, recall, f1, auroc = evaluate_models(shard_models_unlearned, test_loader)
# results_unlearning.append((S, R, "Majority Voting" + " (Unlearned)", accuracy, precision, recall, f1, auroc))

# if accuracy > best_accuracy_unlearned:
#     best_accuracy_unlearned = accuracy
#     best_model_unlearned = shard_models_unlearned
#     best_result_unlearned = (S, R, accuracy, precision, recall, f1, auroc)

# clear_CUDA()

In [21]:
clear_CUDA()

In [22]:
S = 20
R = 5
shards = create_shards(train_dataset, S)
shard_models = sisa_train(shards, R, num_epochs, initial_model, criterion, optimizer_fn)

test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

accuracy, precision, recall, f1, auroc = evaluate_models(shard_models, test_loader)
results.append((S, R, "Majority Voting", accuracy, precision, recall, f1, auroc))

if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_model = shard_models
    best_result = (S, R, accuracy, precision, recall, f1, auroc)

# Perform unlearning
data_to_forget_indices = np.random.choice(len(train_data), 500, replace=False)
shard_models_unlearned = sisa_unlearn(shards, R, shard_models, initial_model, criterion, optimizer_fn, data_to_forget_indices)

accuracy, precision, recall, f1, auroc = evaluate_models(shard_models_unlearned, test_loader)
results_unlearning.append((S, R, "Majority Voting" + " (Unlearned)", accuracy, precision, recall, f1, auroc))

if accuracy > best_accuracy_unlearned:
    best_accuracy_unlearned = accuracy
    best_model_unlearned = shard_models_unlearned
    best_result_unlearned = (S, R, accuracy, precision, recall, f1, auroc)
    
clear_CUDA()

Training shard 1/20, slice 1/5


Epoch 1/5:  88%|████████▊ | 7/8 [00:00<00:00, 31.75it/s]

Epoch 5/5: 100%|██████████| 8/8 [00:00<00:00, 35.02it/s]


Training shard 1/20, slice 2/5


Epoch 5/5: 100%|██████████| 8/8 [00:00<00:00, 35.06it/s]


...

Epoch 5/5: 100%|██████████| 8/8 [00:00<00:00, 35.92it/s]


Slice 4/5 retrained for shard 16/20


Epoch 5/5: 100%|██████████| 8/8 [00:00<00:00, 35.45it/s]


Slice 5/5 retrained for shard 16/20


Evaluating model: Shard 20/20, slice 5/5: 100%|██████████| 157/157 [00:02<00:00, 53.35it/s]


In [23]:
# S = 20
# R = 10
# shards = create_shards(train_dataset, S)
# shard_models = sisa_train(shards, R, num_epochs, initial_model, criterion, optimizer_fn)

# test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# accuracy, precision, recall, f1, auroc = evaluate_models(shard_models, test_loader)
# results.append((S, R, "Majority Voting", accuracy, precision, recall, f1, auroc))

# if accuracy > best_accuracy:
#     best_accuracy = accuracy
#     best_model = shard_models
#     best_result = (S, R, accuracy, precision, recall, f1, auroc)

# # Perform unlearning
# data_to_forget_indices = np.random.choice(len(train_data), 500, replace=False)
# shard_models_unlearned = sisa_unlearn(shards, R, shard_models, initial_model, criterion, optimizer_fn, data_to_forget_indices)

# accuracy, precision, recall, f1, auroc = evaluate_models(shard_models_unlearned, test_loader)
# results_unlearning.append((S, R, "Majority Voting" + " (Unlearned)", accuracy, precision, recall, f1, auroc))


# if accuracy > best_accuracy_unlearned:
#     best_accuracy_unlearned = accuracy
#     best_model_unlearned = shard_models_unlearned
#     best_result_unlearned = (S, R, accuracy, precision, recall, f1, auroc)

# clear_CUDA()

In [24]:
# S = 20
# R = 20
# shards = create_shards(train_dataset, S)
# shard_models = sisa_train(shards, R, num_epochs, initial_model, criterion, optimizer_fn)

# test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# accuracy, precision, recall, f1, auroc = evaluate_models(shard_models, test_loader)
# results.append((S, R, "Majority Voting", accuracy, precision, recall, f1, auroc))

# if accuracy > best_accuracy:
#     best_accuracy = accuracy
#     best_model = shard_models
#     best_result = (S, R, accuracy, precision, recall, f1, auroc)

# # Perform unlearning
# data_to_forget_indices = np.random.choice(len(train_data), 500, replace=False)
# shard_models_unlearned = sisa_unlearn(shards, R, shard_models, initial_model, criterion, optimizer_fn, data_to_forget_indices)

# accuracy, precision, recall, f1, auroc = evaluate_models(shard_models_unlearned, test_loader)
# results_unlearning.append((S, R, "Majority Voting" + " (Unlearned)", accuracy, precision, recall, f1, auroc))

# if accuracy > best_accuracy_unlearned:
#     best_accuracy_unlearned = accuracy
#     best_model_unlearned = shard_models_unlearned
#     best_result_unlearned = (S, R, accuracy, precision, recall, f1, auroc)

# clear_CUDA()

## Results for Q1:

In [25]:
df_results = pd.DataFrame(results, columns=['S', 'R', 'Aggregation Method', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUROC'])
print(df_results)

    S   R Aggregation Method  Accuracy  Precision  Recall  F1-Score     AUROC
0   5   5    Majority Voting    0.7115   0.718376  0.7115  0.713731  0.839722
1   5  10    Majority Voting    0.6895   0.692955  0.6895  0.689088  0.827500
2   5  20    Majority Voting    0.6278   0.649094  0.6278  0.618279  0.793222
3  10   5    Majority Voting    0.6887   0.693371  0.6887  0.689312  0.827056
4  10  10    Majority Voting    0.6461   0.658433  0.6461  0.640782  0.803389
5  20   5    Majority Voting    0.6351   0.651874  0.6351  0.628834  0.797278


## Results for Q2:

In [26]:
df_results_unlearned = pd.DataFrame(results_unlearning, columns=['S', 'R', 'Aggregation Method', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUROC'])
print(df_results_unlearned)

    S   R           Aggregation Method  Accuracy  Precision  Recall  F1-Score  \
0   5   5  Majority Voting (Unlearned)    0.7195   0.720219  0.7195  0.719193   
1   5  10  Majority Voting (Unlearned)    0.6906   0.697952  0.6906  0.690760   
2   5  20  Majority Voting (Unlearned)    0.6383   0.651818  0.6383  0.630776   
3  10   5  Majority Voting (Unlearned)    0.6890   0.691466  0.6890  0.689051   
4  10  10  Majority Voting (Unlearned)    0.6428   0.656275  0.6428  0.636842   
5  20   5  Majority Voting (Unlearned)    0.6341   0.650100  0.6341  0.627639   

      AUROC  
0  0.844167  
1  0.828111  
2  0.799056  
3  0.827222  
4  0.801556  
5  0.796722  


# 1.3

In [27]:

def calculate_losses(model, dataloader):
    
    model.eval()
    losses = []
    criterion = nn.CrossEntropyLoss(reduction='none')
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            losses.extend(loss.cpu().numpy())
    return np.array(losses)


In [28]:
def membership_inference_attack(losses_train, losses_test):
    X = np.concatenate([losses_train, losses_test]).reshape(-1, 1)
    y = np.concatenate([np.ones(len(losses_train)), np.zeros(len(losses_test))])
    model = LogisticRegression()
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return scores.mean()


In [29]:
all_models = []
for shard_idx in best_model.keys():
    all_models.extend(best_model[shard_idx])

all_models_unlearning = []
for shard_idx in best_model_unlearned.keys():
    all_models_unlearning.extend(best_model_unlearned[shard_idx])


In [32]:
clear_CUDA()

In [33]:
# Randomly select data points to forget

num_to_forget = 500
indices_to_forget = random.sample(range(len(train_dataset)), num_to_forget)
retrain_indices = list(set(range(len(train_dataset))) - set(indices_to_forget))


In [34]:

# DataLoader for different sets
retrain_set = Subset(train_dataset, retrain_indices)
forget_set = Subset(train_dataset, indices_to_forget)

train_loader = data.DataLoader(retrain_set, batch_size=64, shuffle=False)
forget_loader = data.DataLoader(forget_set, batch_size=64, shuffle=False)

random_test_indices1 = random.sample(range(len(test_dataset)), num_to_forget)
test_loader_subset1 = data.DataLoader(data.Subset(test_dataset, random_test_indices1), batch_size=64, shuffle=False)

random_test_indices2 = random.sample(range(len(test_dataset)), num_to_forget)
test_loader_subset2 = data.DataLoader(data.Subset(test_dataset, random_test_indices2), batch_size=64, shuffle=False)


In [35]:

# Calculate losses

losses_train = np.concatenate([calculate_losses(model,  train_loader) for model in all_models])
losses_forget = np.concatenate([calculate_losses(model,  forget_loader) for model in all_models])
losses_test1 = np.concatenate([calculate_losses(model,  test_loader_subset1) for model in all_models])
losses_test2 = np.concatenate([calculate_losses(model,  test_loader_subset2) for model in all_models])

losses_train_unlearned = np.concatenate([calculate_losses(model,  train_loader) for model in all_models_unlearning])
losses_forget_unlearned = np.concatenate([calculate_losses(model,  forget_loader) for model in all_models_unlearning])
losses_test1_unlearned  = np.concatenate([calculate_losses(model,  test_loader_subset1) for model in all_models_unlearning])
losses_test2_unlearned  = np.concatenate([calculate_losses(model,  test_loader_subset2) for model in all_models_unlearning])


In [36]:

# Perform MIA
score_trained_1_2 = membership_inference_attack(losses_train, losses_test1)
score_trained_3_4 = membership_inference_attack(losses_forget, losses_test2)
score_unlearned_1_2 = membership_inference_attack(losses_train_unlearned, losses_test1_unlearned)
score_unlearned_3_4 = membership_inference_attack(losses_forget_unlearned, losses_test2_unlearned)


## Results for Q3:

In [37]:

# Report Results
results = {
    "Trained Model (Not Unlearned) - Dataset retrain vs test1": score_trained_1_2,
    "Trained Model (Not Unlearned) - Dataset forget vs test2": score_trained_3_4,
    "Unlearned Model - Dataset retrain vs test1": score_unlearned_1_2,
    "Unlearned Model - Dataset forget vs test2": score_unlearned_3_4,
}


results

{'Trained Model (Not Unlearned) - Dataset retrain vs test1': 0.99,
 'Trained Model (Not Unlearned) - Dataset forget vs test2': 0.518,
 'Unlearned Model - Dataset retrain vs test1': 0.99,
 'Unlearned Model - Dataset forget vs test2': 0.5109600000000001}